In [1]:
import pandas as pd
from transformers import AutoTokenizer, Trainer, TrainingArguments, DataCollatorForLanguageModeling, LlamaForCausalLM, LlamaConfig, TrainerCallback
import torch
from torch.utils.data import Dataset, DataLoader
import math

In [ ]:
import pandas as pd
from transformers import AutoTokenizer, Trainer, TrainingArguments, DataCollatorForLanguageModeling, LlamaForCausalLM, LlamaConfig, TrainerCallback
import torch
from torch.utils.data import Dataset, DataLoader
import math




import pandas as pd
from transformers import AutoTokenizer
import torch
import gc
from tqdm import tqdm

# Load Bengali Tokenizer
print("Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained('/kaggle/input/eos-tokenizer')

# Load and preprocess Bengali text data from CSV in chunks
print("Loading and preprocessing text data from CSV...")
chunk_size = 1000  # Adjust based on available memory
context_length = 256

# Define a generator function to read and process chunks with a progress bar
def text_generator(file_path, tokenizer, context_length, chunk_size=1000):
    total_lines = sum(1 for _ in open(file_path)) - 1  # Total lines excluding header
    with tqdm(total=total_lines, desc="Processing CSV") as pbar:
        for chunk in pd.read_csv(file_path, chunksize=chunk_size):
            chunk['text'] = chunk['text'].apply(str)
            chunk['text'] = chunk['text'].apply(lambda x: f"{tokenizer.bos_token} {x} {tokenizer.eos_token}")
            
            # Tokenize each text in the chunk
            for text in chunk['text']:
                tokens = tokenizer.encode(text, add_special_tokens=True)
                for i in range(0, len(tokens), context_length):
                    pbar.update(1)  # Update the progress bar
                    yield tokens[i: i + context_length]

# Create a generator instance
tokens_generator = text_generator('/kaggle/input/bengali-news/news.csv', tokenizer, context_length)

# Convert the generator to a list (optional, only if you need to store it)
chunks = list(tokens_generator)

# Clear any unused memory
torch.cuda.empty_cache()
gc.collect()

print("Tokenization complete. Total chunks created:", len(chunks))


Loading tokenizer...
Loading and preprocessing text data from CSV...


Processing CSV: 143295it [01:58, 1213.65it/s]                          


Tokenization complete. Total chunks created: 143295


In [3]:
print(chunks[0])

[2, 11798, 9634, 9635, 1142, 7608, 5300, 2019, 1261, 14146, 1089, 3193, 8647, 13965, 26, 10156, 2627, 514, 17880, 4441, 7954, 1524, 1317, 15981, 4198, 3438, 6226, 1102, 1742, 2332, 13454, 14146, 1022, 1460, 4329, 2859, 2978, 1114, 3607, 2442, 552, 1246, 10077, 18685, 812, 1039, 23680, 1056, 1220, 4846, 6506, 523, 576, 4329, 545, 1805, 3268, 10770, 28532, 5677, 3073, 3456, 1317, 15981, 1799, 1027, 1442, 1742, 2332, 7640, 1116, 1410, 1742, 1241, 13454, 24674, 1317, 15981, 4198, 3438, 6226, 1102, 1742, 2332, 7640, 1116, 1442, 1742, 1241, 7640, 26, 14146, 1022, 3242, 1460, 4329, 2859, 2978, 3356, 19050, 10156, 2627, 14146, 9959, 20719, 4362, 2035, 2172, 7548, 1517, 1316, 1340, 2896, 14146, 1012, 4445, 1895, 26286, 1260, 1943, 20566, 13311, 1542, 15981, 7938, 7947, 1164, 1057, 7877, 7293, 5978, 7081, 1496, 1007, 3406, 19778, 10770, 24156, 514, 1133, 1967, 1375, 7648, 3883, 11430, 16074, 8067, 1402, 26, 4198, 3438, 7938, 1539, 20350, 15289, 1284, 14146, 1384, 1460, 4329, 545, 514, 15549, 427

In [4]:


# Dataset Class
print("Creating dataset class...")
class BengaliDataset(Dataset):
    def __init__(self, chunks):
        self.chunks = chunks

    def __len__(self):
        return len(self.chunks)

    def __getitem__(self, idx):
        input_ids = torch.tensor(self.chunks[idx], dtype=torch.long)
        attention_mask = torch.ones(len(input_ids), dtype=torch.long)  # All tokens are attended to

        # Padding
        padding_length = context_length - input_ids.size(0)
        if padding_length > 0:
            input_ids = torch.cat([input_ids, torch.zeros(padding_length, dtype=torch.long)])
            attention_mask = torch.cat([attention_mask, torch.zeros(padding_length, dtype=torch.long)])

        return {'input_ids': input_ids, 'attention_mask': attention_mask}

# Create Dataset
print("Splitting dataset into train and validation sets...")
bengali_dataset = BengaliDataset(chunks)
train_size = int(0.8 * len(bengali_dataset))
val_size = len(bengali_dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(bengali_dataset, [train_size, val_size])




def inspect_dataset(dataset, tokenizer, num_samples=5):
    for i in range(num_samples):
        sample = dataset[i]
        input_ids = sample['input_ids']
        attention_mask = sample['attention_mask']
        
        print(f"\nSample {i+1}:")
        print(f"Length: {len(input_ids)}")
        print(f"Non-padding tokens: {attention_mask.sum()}")
        print(f"Text: {tokenizer.decode(input_ids)}")

# Then call it on both datasets:
print("\nInspecting Training Dataset:")
inspect_dataset(train_dataset, tokenizer)

print("\nInspecting Validation Dataset:")
inspect_dataset(val_dataset, tokenizer)

Creating dataset class...
Splitting dataset into train and validation sets...

Inspecting Training Dataset:

Sample 1:
Length: 256
Non-padding tokens: 256
Text: করেন। তিনি বলেন, “প্রতিদিন প্রায় ৭০-৮০ কাপ মটকা চা বিক্রি করা হয়। তবে ছুটির দিন হলে বিক্রি ১০০ কাপ ছাড়িয়ে যায়। শীত মৌসুমে বেচাকেনা বেড়ে যায়।”বাবুর ভাষ্য, “কুষ্টিয়া জেলায় আমিই একমাত্র মটকা চা বিক্রি করি। মটকা চা খেতে প্রায় প্রতিদিনই অন্য জেলা থেকেও চা পিপাসুরা আসেন।”তার দোকানে মটকা চা তৈরিতে দুধ-চিনি ছাড়া আর কোনো মসলা ব্যবহার করা হয় না। প্রতি কাপ চায়ের দাম রাখেন ৩৫ টাকা। এছাড়া বাবুর দোকানে দুধ চা, লাল চা, মধু চা-ও পাওয়া যায়। কুষ্টিয়ার বাসিন্দা বহুজাতিক প্রতিষ্ঠান ওয়ালটনের ন্যাশনাল সেলস ম্যানেজার ওয়ালিউল ইসলাম আজিম। তিনিও এই মটকা চায়ের ভক্ত।আজিম বলেন, “আমি ঢাকা থেকে কুষ্টিয়া এলে সময় থাকলেই মটকা চা খাওয়ার জন্য বাবুর দোকানে যাওয়া হয়। এই চায়ের স্বাদ অতুলনীয়।”স্থানীয় সাগরখালী আদর্শ ডিগ্রি কলেজের প্রভাষক মশিউর রহমান বলেন, “আমি প্রায়ই মটকা চা খেতে এখানে আসি। মটকা চায়ের বৈশিষ্ট্য হলো এক কেজি দুধে মাত্র পাঁচ

In [5]:
# Model Configuration
print("Configuring Llama model...")
config = LlamaConfig(
    hidden_size=512,  # Adjust hidden size
    vocab_size=len(tokenizer.vocab),
    num_attention_heads=8,
    num_hidden_layers=8,
    intermediate_size=256,
    eos_token_id=tokenizer.eos_token_id,
    bos_token_id=tokenizer.bos_token_id,
    pad_token_id=tokenizer.pad_token_id
)
model = LlamaForCausalLM(config)

# Calculate and print total parameters
print("Calculating total parameters in the model...")
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Total parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")

# Training Setup
print("Setting up training arguments and device...")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

training_args = TrainingArguments(
    output_dir="./bengali_llama",
    evaluation_strategy="steps",
    eval_steps=1100,  # Evaluate every 500 steps
    logging_steps=100,
    per_device_train_batch_size=10,
    per_device_eval_batch_size=10,
    save_steps=1100,
    save_total_limit=1,
    report_to="none",
    num_train_epochs=6,
    do_train=True,
    do_eval=True
)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)


import math
# import wandb
from transformers import TrainerCallback

class PerplexityCallback(TrainerCallback):
    def __init__(self):
        self.perplexity_by_epoch = []  # List to store [epoch, perplexity] pairs

    def on_evaluate(self, args, state, control, metrics=None, **kwargs):
        if metrics and "eval_loss" in metrics:
            perplexity = math.exp(metrics["eval_loss"])
            # Store [epoch, perplexity] in the matrix
            self.perplexity_by_epoch.append([state.epoch, perplexity])
            # Log the perplexity to W&B
            print({"eval_perplexity": perplexity,"epoch":state.epoch})

    def on_log(self, args, state, control, logs=None, **kwargs):
        if "eval_loss" in logs:
            perplexity = math.exp(logs["eval_loss"])
            # Log the perplexity to W&B
            # wandb.log({"eval_perplexity": perplexity})
            print({"eval_perplexity": perplexity,"epoch":state.epoch})

    def on_train_end(self, args, state, control, **kwargs):
        # Print the matrix after training is done
        print("Epoch-Perplexity Matrix:")
        for epoch, perplexity in self.perplexity_by_epoch:
            print(f"Epoch: {epoch}, Perplexity: {perplexity}")






# Trainer with Perplexity Callback
print("Initializing Trainer...")
perplexity_callback = PerplexityCallback()
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    callbacks=[PerplexityCallback]
)

# Training
print("Starting training...")
trainer.train()

# Save Model and Tokenizer
print("Saving model and tokenizer...")
model.save_pretrained('/kaggle/input/outpu6')
tokenizer.save_pretrained('/kaggle/input/outpu6')
print("Training complete.")
# print("Perplexity Matrix:", perplexity_callback.perplexity_matrix)

Configuring Llama model...
Calculating total parameters in the model...
Total parameters: 45,098,496
Trainable parameters: 45,098,496
Setting up training arguments and device...


/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Initializing Trainer...
Starting training...


Step,Training Loss,Validation Loss
1100,8.161500,8.111774
2200,6.983900,6.993573
3300,6.466500,6.410713
4400,6.049200,6.037925
5500,5.782000,5.763635
6600,5.546000,5.548570
7700,5.375000,5.376654
8800,5.236600,5.235680
9900,5.144900,5.123484
11000,5.032000,5.018804


{'eval_perplexity': 3333.4878744890184, 'epoch': 0.09595254710397767}
{'eval_perplexity': 3333.4878744890184, 'epoch': 0.09595254710397767}
{'eval_perplexity': 1089.6079033185829, 'epoch': 0.19190509420795535}
{'eval_perplexity': 1089.6079033185829, 'epoch': 0.19190509420795535}
{'eval_perplexity': 608.3273829209174, 'epoch': 0.28785764131193303}
{'eval_perplexity': 608.3273829209174, 'epoch': 0.28785764131193303}
{'eval_perplexity': 419.0227621461282, 'epoch': 0.3838101884159107}
{'eval_perplexity': 419.0227621461282, 'epoch': 0.3838101884159107}
{'eval_perplexity': 318.5040397288412, 'epoch': 0.47976273551988835}
{'eval_perplexity': 318.5040397288412, 'epoch': 0.47976273551988835}
{'eval_perplexity': 256.8698866996486, 'epoch': 0.5757152826238661}
{'eval_perplexity': 256.8698866996486, 'epoch': 0.5757152826238661}
{'eval_perplexity': 216.2973644314433, 'epoch': 0.6716678297278437}
{'eval_perplexity': 216.2973644314433, 'epoch': 0.6716678297278437}
{'eval_perplexity': 187.856735378213

OSError: [Errno 30] Read-only file system: '/kaggle/input/outpu6/config.json'

In [38]:
# Testing Model Outputs for Bengali Prompts


print("Saving model and tokenizer...")
model.save_pretrained('/kaggle/working/outpu6')
tokenizer.save_pretrained('/kaggle/working/outpu6/tokenser')
print("Training complete.")
print("Testing model outputs for Bengali prompts...")
prompts = [
    "কৃষকরা অনেক সমস্যার সম্মুখীন হয়",
    "অনেক উন্নয়ন কর্মকান্ড যেমন",
    "বঙ্গবন্ধু শেখ মুজিবুর রহমান",
    "বাংলাদেশের ক্রিকেট দলের সাম্প্রতিক পারফরম্যান্স কেমন",
    "একটি সূর্যাস্তের বর্ণনা দাও",
    "মহাসড়কে মোটরসাইকেল চলাচল ",
    "ভারতের উত্তর প্রদেশে এক বিয়ের অনুষ্ঠানে ",
    "পাবলো মারির ওপর হামলা",
    "র‌্যাবসহ ৫৯ জনকে উদ্ধার করেছে",
    "এলএনজি আমদানি পুনরায় শুরু এবং বিকল্প জ্বালানির"
]


for i, prompt in enumerate(prompts):
    print(f"Generating text for prompt {i+1}: {prompt}")
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    inputs.pop("token_type_ids", None)

    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=200,do_sample=True,temperature=0.2)
    # print(outputs)
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    print(f"Prompt {i+1}: {prompt}")
    print(f"Output {i+1}: {generated_text}\n")

Saving model and tokenizer...
Training complete.
Testing model outputs for Bengali prompts...
Generating text for prompt 1: কৃষকরা অনেক সমস্যার সম্মুখীন হয়
Prompt 1: কৃষকরা অনেক সমস্যার সম্মুখীন হয়
Output 1: কৃষকরা অনেক সমস্যার সম্মুখীন হয় না।”তিনি আরও বলেন, “এখন পর্যন্ত আমরা প্রায় ১০ লাখ মানুষ বিদ্যুৎ উৎপাদন করতে সক্ষম হই।” 

Generating text for prompt 2: অনেক উন্নয়ন কর্মকান্ড যেমন
Prompt 2: অনেক উন্নয়ন কর্মকান্ড যেমন
Output 2: অনেক উন্নয়ন কর্মকান্ড যেমন হয় তেমনি তাদেরও হয়রানি করা হয় না।”তিনি বলেন, “এখন পর্যন্ত আমাদের দেশে প্রায় ১০ লাখ মানুষ বসবাস করছে। এর মধ্যে প্রায় ১০ লাখ মানুষ বসবাস করছে।” 

Generating text for prompt 3: বঙ্গবন্ধু শেখ মুজিবুর রহমান
Prompt 3: বঙ্গবন্ধু শেখ মুজিবুর রহমান
Output 3: বঙ্গবন্ধু শেখ মুজিবুর রহমান বিজ্ঞান ও প্রযুক্তি বিশ্ববিদ্যালয়ের উপাচার্য অধ্যাপক ড. মো. আখতারুজ্জামানের সভাপতিত্বে অনুষ্ঠিত সভায় আরও বক্তব্য রাখেন বিশ্ববিদ্যালয়ের উপাচার্য অধ্যাপক ড. মো. আখতারুজ্জামান। 

Generating text for prompt 4: বাংলাদেশের ক্রিকেট দলের সাম্প্রতিক পারফরম

In [7]:
def generate_text(prompt, model, tokenizer, device, 
                 max_length=150,
                 temperature=0.8,
                 top_p=0.9,
                 top_k=50):
    """Generate text with controlled parameters"""
    inputs = tokenizer(prompt, 
                     return_tensors="pt",
                     padding=True,
                     truncation=True,
                     max_length=512).to(device)
    
    inputs.pop("token_type_ids", None)
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_length=max_length,
            temperature=temperature,
            top_p=top_p,
            top_k=top_k,
            do_sample=True,
            pad_token_id=tokenizer.pad_token_id,
            bos_token_id=tokenizer.bos_token_id,
            eos_token_id=tokenizer.eos_token_id,
            repetition_penalty=1.2
        )
    
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Different parameter combinations to test
parameter_sets = [
    {"max_length": 30, "temperature": 0.6},   # Very conservative
    {"max_length": 50, "temperature": 0.7},   # Conservative
    {"max_length": 100, "temperature": 0.8},  # Balanced
    {"max_length": 150, "temperature": 0.9},  # Creative
    {"max_length": 200, "temperature": 1.0},  # Very creative
]

# Diverse prompts across different categories
prompts = [
    # Short Questions/Conversations
    "তোমার নাম কি?",
    "কেমন আছো?",
    "আজ কি খেয়েছো?",
    
    # Personal Descriptions
    "নিজের সম্পর্কে কিছু বলো",
    "তোমার পছন্দের রং কি এবং কেন?",
    "তোমার প্রিয় খেলা কি?",
    
    # Creative Writing Prompts
    "একটি সূর্যাস্তের বর্ণনা দাও",
    "বর্ষার প্রথম দিনের গল্প লেখো",
    "একটি প্রেমের কবিতা লেখো",
    
    # Academic/Educational
    "বাংলাদেশের স্বাধীনতা যুদ্ধ সম্পর্কে লেখো",
    "গ্লোবাল ওয়ার্মিং কি এবং এর প্রভাব কি?",
    "রবীন্দ্রনাথ ঠাকুরের সাহিত্যিক অবদান কি?",
    
    # Instructions/How-to
    "ইলিশ মাছ রান্নার রেসিপি লেখো",
    "গাড়ি চালানো শেখার টিপস দাও",
    "ফেসবুক অ্যাকাউন্ট খোলার নিয়ম বলো",
    
    # Opinion/Analysis
    "সোশ্যাল মিডিয়ার প্রভাব কি?",
    "আধুনিক শিক্ষা ব্যবস্থার ভালো-মন্দ দিক আলোচনা করো",
    "জলবায়ু পরিবর্তন মোকাবেলায় আমাদের করণীয় কি?",
    
    # Business/Professional
    "একটি জব ইন্টারভিউ এর প্রস্তুতি কিভাবে নেবেন?",
    "স্টার্টআপ শুরু করার জন্য কি কি প্রয়োজন?",
    "একটি প্রফেশনাল ইমেইল লেখার নিয়ম কি?",
    
    # Cultural/Traditional
    "পহেলা বৈশাখ উদযাপন কিভাবে করা হয়?",
    "বাংলাদেশের ঐতিহ্যবাহী খাবার গুলো কি কি?",
    "শুভ বিবাহের আচার-অনুষ্ঠান গুলো কি কি?",
    
    # Scientific/Technical
    "কৃত্রিম বুদ্ধিমত্তা কিভাবে কাজ করে?",
    "ইন্টারনেট কিভাবে কাজ করে?",
    "সৌরজগৎ সম্পর্কে একটি বৈজ্ঞানিক ব্যাখ্যা দাও",
    
    # News/Current Affairs
    "বর্তমান বিশ্বের প্রধান সমস্যাগুলো কি কি?",
    "ডিজিটাল বাংলাদেশের অগ্রগতি কেমন?",
    "করোনা মহামারী থেকে কি শিক্ষা পেলাম?",
    
    # Health/Medical
    "স্বাস্থ্যকর জীবনযাপনের নিয়ম কি কি?",
    "ডায়াবেটিস প্রতিরোধে করণীয় কি?",
    "মানসিক স্বাস্থ্য রক্ষার উপায় কি?",

       # **Basic QA (Question Answering)**
    "বাংলাদেশের জাতীয় ফুল কী?",
    "নদীর নামের মধ্যে ‘গঙ্গা’ কোথায় অবস্থিত?",
    "বিশ্ববিদ্যালয় কবে প্রতিষ্ঠিত হয়েছিল?",
    "কেন মধু মাখার জন্য মৌমাছি প্রয়োজন?",
    
    # **Completion Prompts**
    "এই লেখাটি সম্পূর্ণ করুন: 'একটি শিশুর সবচেয়ে ভালো বন্ধু হল...'",
    "শেষ বাক্যটি পূর্ণ করুন: 'এটি একটি সুন্দর সকাল, আকাশে...'",
    
    # **Explain**
    "ভোকাল কর্ড কী এবং এটি কিভাবে কাজ করে?",
    "সংবিধান কী এবং এর গুরুত্ব কি?",
    "ডিজিটাল মার্কেটিং কী এবং এর বিভিন্ন উপাদান কি?",
    
    # **Conversational Prompts**
    "আপনি কি কখনো সঙ্গীত শুনতে ভালোবাসেন? আপনার প্রিয় গান কী?",
    "আজকাল আপনি কীভাবে আপনার সময় কাটান? আমার সাথে কিছু শেয়ার করতে চান?",
    
    # **Creative Writing Prompts**
    "একটি কাল্পনিক শহরের বর্ণনা দিন যেখানে প্রযুক্তি এবং প্রকৃতি একসাথে কাজ করে।",
    "একটি রোমাঞ্চকর উপন্যাসের শুরুর অংশ লিখুন যেখানে একজন অধ্যাপক রহস্যময় বই আবিষ্কার করেন।",
    "বৃষ্টির দিনে একমাত্র স্কুলে যাওয়া ছাত্রটির অনুভূতি বর্ণনা করুন।",
    
    # **Task-Oriented Prompts**
    "আজকের আবহাওয়া যদি বৃষ্টি হয়, তবে আমাকে কী প্রস্তুতি নেওয়া উচিত?",
    "আমি কীভাবে একটি ভাল রোজগারের পরিকল্পনা তৈরি করতে পারি?",
    
    # **Philosophical/Reflective Prompts**
    "জীবনের অর্থ কী হতে পারে?",
    "আপনার জীবনের সবচেয়ে বড় শিক্ষা কী?",
    
    # **Explanatory Tasks**
    "আপনি কীভাবে একটি সাধারণ প্রোগ্রামিং ভাষা শিখতে পারেন?",
    "কম্পিউটার সিস্টেমে RAM এবং ROM এর মধ্যে পার্থক্য ব্যাখ্যা করুন।",
    
    # **Opinion-Based Prompts**
    "আপনি কী মনে করেন, মানুষের জন্য প্রযুক্তি কতটা উপকারী বা ক্ষতিকর?",
    "আপনার মতে, পড়াশোনা শেষ করার পর জীবনের প্রথম সিদ্ধান্ত কী হওয়া উচিত?",
    
    # **Fact-Based Prompts**
    "নেপচুন গ্রহের বিশেষত্ব কী?",
    "আইফেল টাওয়ার কবে এবং কোথায় নির্মিত হয়েছিল?",
    
    # **Descriptive Prompts**
    "একটি সরিষাক্ষেতের দৃশ্য বর্ণনা করুন যা সারা পৃথিবী জুড়ে বিখ্যাত।",
    "একটি শান্তিপূর্ণ গ্রামের একটি সকালে হেঁটে যাওয়া ব্যক্তির অভিজ্ঞতা বর্ণনা করুন।",
    
    # **Comparative Prompts**
    "বাঙালি খাবার এবং ভারতীয় খাবারের মধ্যে কী কী পার্থক্য আছে?",
    "ইউরোপীয় সংস্কৃতি এবং এশীয় সংস্কৃতির মধ্যে প্রধান পার্থক্য কী?",
    
    # **Jokes or Light Conversation**
    "একটি মজার গল্প বলুন।",
    "আপনার কাছে কোনো মজার কৌতুক থাকলে তা শেয়ার করুন।",
    
    # **Role-Playing Prompts**
    "আপনি যদি একটি রেস্তোরাঁর ম্যানেজার হতেন, তবে আপনার প্রথম কাজ কী হতো?",
    "আপনি যদি একটি রহস্য সমাধানকারী detective হতেন, আপনার প্রথম তদন্ত কী হতো?",
    
    # **Hypothetical Situation**
    "আপনি যদি ১০০ কোটি টাকা পেতেন, তবে আপনি কীভাবে তা ব্যয় করতেন?",
    "যদি আপনি কোনো সময়ে ফিরে যেতে পারতেন, তবে আপনি কোথায় যেতেন?",
    
    # **Advice Requests**
    "কীভাবে নিজেকে একটি নতুন পরিবেশে মানিয়ে নিতে পারেন?",
    "কীভাবে আত্মবিশ্বাস তৈরি করবেন?"
]

print("Testing model outputs with different parameters...\n")

for i, prompt in enumerate(prompts, 1):
    print(f"\nPrompt {i}: {prompt}")
    print("=" * 80)  # Separator
    
    # Test each parameter combination
    for params in parameter_sets:
        print(f"\nParameters: max_length={params['max_length']}, temperature={params['temperature']}")
        print("-" * 40)
        
        generated_text = generate_text(
            prompt=prompt,
            model=model,
            tokenizer=tokenizer,
            device=device,
            max_length=params['max_length'],
            temperature=params['temperature']
        )
        
        print(f"Output: {generated_text}")
        # Print token count for comparison
        token_count = len(tokenizer.encode(generated_text))
        print(f"Token count: {token_count}")
        print("-" * 80)

    print("\n" + "=" * 80 + "\n")  # Separator between prompts

Testing model outputs with different parameters...


Prompt 1: তোমার নাম কি?

Parameters: max_length=30, temperature=0.6
----------------------------------------
Output: তোমার নাম কি? এটা আমার কাছে মনে হয়েছে যে আমি সবসময় আছি।”“আমি আমার বিশ্বাসকে বলতে চাই, “এটা আমাকে দেখে বলেছি,
Token count: 29
--------------------------------------------------------------------------------

Parameters: max_length=50, temperature=0.7
----------------------------------------
Output: তোমার নাম কি?”এর আগে গত সপ্তাহে “আদিপুরুষ” সিনেমাটি নির্মাণ করা হয়েছিল। কিন্তু সেই ছবিটি দর্শককে সামনে রেখে সিনেমাটিকে “পাঠান” বলে সম্বোধন করে দর্শকরা। তবে তা আর হয়নি। 
Token count: 41
--------------------------------------------------------------------------------

Parameters: max_length=100, temperature=0.8
----------------------------------------
Output: তোমার নাম কি?' যোগ করেন তিনি। 
Token count: 8
--------------------------------------------------------------------------------

Parameters: max_length=150, temperature